# Invoke-WebRequest

**Objective**: Scrape data from a webpage without an API service

**Tutorial**: [Web Scraping with PowerShell](https://www.pipehow.tech/invoke-webscrape/)

**Website**: [Quotes to Scrape](http://quotes.toscrape.com/)

**Data to Capture / Parse:**

-   Quote
-   Author
-   Tags

![](https://github.com/MergeWords/PowerShell/blob/Invoke-WebRequest/Invoke-WebRequest/Images/screencapture-quotes-toscrape-2020-06-12-02_16_10-edit.png?raw=true)

## Step 1 - Inspect HTML structure

In [ ]:
$WebSite = Invoke-WebRequest "http://quotes.toscrape.com/"
$WebSite

## Step 2 - Use the **Content** method to view all the HTML for the site and pick out the elements needed to parse Quote, Author, and Tags data

In [ ]:
$Website.Content

-   **Quote** is in the [span](https://developer.mozilla.org/en-US/docs/Web/HTML/Element/span) element
-   **Author** is in the [small](https://developer.mozilla.org/en-US/docs/Web/HTML/Element/small) element
-   **Tags** are in the [meta](https://developer.mozilla.org/en-US/docs/Web/HTML/Element/meta) element


<mark>`<span class`</mark>`="text" itemprop="text">"The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking."</span>`

`<span>by `<mark>`<small class`</mark>`="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>`

`<div class="tags">
    Tags:
    `<mark>`<meta class`</mark>`="keywords" itemprop="keywords" content="change,deep-thoughts,thinking,world" /    >`